In [7]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from tqdm import tqdm

In [8]:
DATA_PATH = Path("../data/user_item_dl.csv")   # Path to your dataset
df = pd.read_csv(DATA_PATH)
print("Data shape:", df.shape)
print(df.head())

Data shape: (268285, 2)
   user_id                              item_id
0  17850.0   white hanging heart t-light holder
1  17850.0                  white metal lantern
2  17850.0       cream cupid hearts coat hanger
3  17850.0  knitted union flag hot water bottle
4  17850.0       red woolly hottie white heart.


In [9]:
# Convert user and item IDs to indices
user2idx = {u: i for i, u in enumerate(df["user_id"].unique())}
item2idx = {it: j for j, it in enumerate(df["item_id"].unique())}   

df["user_idx"] = df["user_id"].map(user2idx)
df["item_idx"] = df["item_id"].map(item2idx)

n_users, n_items = len(user2idx), len(item2idx)
print(f"n_users = {n_users}, n_items = {n_items}")

n_users = 4338, n_items = 3866


In [10]:
# dataset for training
class InteractDS(Dataset):
    def __init__(self, frame: pd.DataFrame):
        self.u = torch.tensor(frame["user_idx"].values, dtype=torch.long)
        self.i = torch.tensor(frame["item_idx"].values, dtype=torch.long)
    def __len__(self): return len(self.u)
    def __getitem__(self, idx):
        return self.u[idx], self.i[idx]

BATCH = 1024
loader = DataLoader(InteractDS(df), batch_size=BATCH, shuffle=True)

In [11]:
# Define the NCF model
class NCF(nn.Module):
    def __init__(self, n_users: int, n_items: int, emb_size: int = 64):
        super().__init__()
        self.u_emb = nn.Embedding(n_users, emb_size)
        self.i_emb = nn.Embedding(n_items, emb_size)
        self.fc = nn.Sequential(
            nn.Linear(emb_size * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
    def forward(self, u, i):
        x = torch.cat([self.u_emb(u), self.i_emb(i)], dim=1)
        return self.fc(x).squeeze()

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model  = NCF(n_users, n_items).to(device)
opt    = torch.optim.Adam(model.parameters(), lr=1e-3)
lossf  = nn.BCELoss()

EPOCHS = 10
for epoch in range(1, EPOCHS + 1):
    model.train(); running = 0
    for u, i in tqdm(loader, desc=f"Epoch {epoch}/{EPOCHS}"):
        u, i = u.to(device), i.to(device)
        pred = model(u, i)
        # implicit feedback: mọi tương tác = 1
        loss = lossf(pred, torch.ones_like(pred))
        opt.zero_grad(); loss.backward(); opt.step()
        running += loss.item() * len(u)
    print(f"Epoch {epoch}: loss = {running/len(df):.4f}")

Epoch 1/10: 100%|██████████| 262/262 [00:02<00:00, 110.75it/s]


Epoch 1: loss = 0.0819


Epoch 2/10: 100%|██████████| 262/262 [00:02<00:00, 105.52it/s]


Epoch 2: loss = 0.0021


Epoch 3/10: 100%|██████████| 262/262 [00:02<00:00, 93.59it/s] 


...

Epoch 8: loss = 0.0001


Epoch 9/10: 100%|██████████| 262/262 [00:03<00:00, 71.76it/s]


Epoch 9: loss = 0.0001


Epoch 10/10: 100%|██████████| 262/262 [00:03<00:00, 71.90it/s]

Epoch 10: loss = 0.0000


In [13]:
MODEL_PATH = Path("../models/ncf_model.pt")
MODEL_PATH.parent.mkdir(exist_ok=True)
torch.save({
    "model": model.state_dict(),
    "user2idx": user2idx,
    "item2idx": item2idx
}, MODEL_PATH)
print("Saved model →", MODEL_PATH.resolve())

Saved model → D:\Documents\My Project\FP-Growth_mba\models\ncf_model.pt
